In [ ]:
import wikipediaapi

# Overview
1. Gather a list of each company in the S&P 500
   1. From Wikipedia
      1. Company Name
      2. Ticker
      3. Headquarters Location
      4. GCIS Sub-Industry
      5. Year Founded
   2. From 

In [ ]:
# Check if a page exists
wiki_wiki = wikipediaapi.Wikipedia('Joeys company guesser project', 'en')

# Save the contents of the page within a variable
page_py = wiki_wiki.page('List of S&P 500 companies')

# Save the contents of the page as text
page_text = page_py.text

def print_sections(sections, level=0):
    """ Gather all of the top level sections within a particular wikipedia page

    Args:
        sections (str): _description_
        level (int, optional): _description_. Defaults to 0.
    """
    for s in sections:
        # if "S&P 500 component stocks" in s.title:
            # print(s.title)
            # print(s.text[0:200])
        print("%s: %s - %s" % ("*" * (level + 1), s.title, s.text[0:4000]))
        print_sections(s.sections, level + 1)
print_sections(page_py.sections)

In [ ]:
# print("Page - Title: %s" % page_py.title)

# print("Page - Summary: %s" % page_py.summary[0:6090])
# print(page_py.text)

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='Joeys company guesser project',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

# Beautiful Soup Approach

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Get the page's HTML content
URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(URL)

# Convert the table to a Pandas DataFrame
df = pd.read_html(response.content, flavor='lxml')[0]

# Display the first few rows of the table
display(df)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


### Find random company

In [2]:
import random
N = len(df)
for i in range(100):
    pick_random_company = random.randint(0,N-1)
    if pick_random_company > 500:
        print(pick_random_company)
        
print(pick_random_company)
display(df.iloc[pick_random_company])

502
27


Symbol                                       AXP
Security                        American Express
GICS Sector                           Financials
GICS Sub-Industry               Consumer Finance
Headquarters Location    New York City, New York
Date added                            1976-06-30
CIK                                         4962
Founded                                     1850
Name: 27, dtype: object

### Get wiki link for each company & append it to the dataframe in a new column

In [31]:
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table
table = soup.find('table', {'class': 'wikitable'})

first_row = table.find_all('tr')[0]
first_link = first_row.find_all('th')[2]

# # print(first_row)
# count = 0
# for row in table.find_all('tr')[1:2]:
#     print(row)
#     print(count)
#     count += 1

# Find all of the links in the second column of the table
links = []
for row in table.find_all('tr')[1:]: # skip the header row
    cells = row.find_all('td')
    if cells:
        link_tag = cells[1].find('a')  # Adjust the index based on the column containing the link
        if link_tag and 'href' in link_tag.attrs:
            links.append(link_tag['href'])

# Create column in existing dataframe with link as element, but with full url
updated_df = df.assign(link=links)
updated_df['wiki_link'] = 'https://en.wikipedia.org' + updated_df['link']
display(updated_df)

# save the dataframe to a CSV file
updated_df.to_csv('S&P_500_companies.csv', index=False)



,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,link,wiki_link
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,/wiki/3M,https://en.wikipedia.org/wiki/3M
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,/wiki/A._O._Smith,https://en.wikipedia.org/wiki/A._O._Smith
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888,/wiki/Abbott_Laboratories,https://en.wikipedia.org/wiki/Abbott_Laboratories
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),/wiki/AbbVie,https://en.wikipedia.org/wiki/AbbVie
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989,/wiki/Accenture,https://en.wikipedia.org/wiki/Accenture
...,...,...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011,/wiki/Xylem_Inc.,https://en.wikipedia.org/wiki/Xylem_Inc.
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997,/wiki/Yum!_Brands,https://en.wikipedia.org/wiki/Yum!_Brands
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969,/wiki/Zebra_Technologies,https://en.wikipedia.org/wiki/Zebra_Technologies
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927,/wiki/Zimmer_Biomet,https://en.wikipedia.org/wiki/Zimmer_Biomet


In [79]:


def get_revenue_from_page(sp500_df):

    # loop through the dataframe and get the revenue for each company
    for index, row in sp500_df.iterrows():
        print(index,row)
        # # Get the HTML content of the linked page
        # link = updated_df.iloc[index]['wiki_link']
        # page_response = requests.get(link)
        # page_soup = BeautifulSoup(page_response.text, 'html.parser')

        # # find the table that include information regarding revenue
        # table = page_soup.find_all('table', {'class': 'infobox ib-company vcard'})
        # for row in table:
        #     revenue = row.find_all('tr')[12]
        #     revenue2 = revenue.find_all('span')[2].text
        #     span_with_title = revenue.find('span').find('span', title=True)
        #     # Extract the title attribute
        #     title = span_with_title['title'] if span_with_title else None
        #     print(title, revenue2[2:])

In [80]:
get_revenue_from_page(updated_df)

0 Symbol                                                MMM
Security                                               3M
GICS Sector                                   Industrials
GICS Sub-Industry                Industrial Conglomerates
Headquarters Location               Saint Paul, Minnesota
Date added                                     1957-03-04
CIK                                                 66740
Founded                                              1902
link                                             /wiki/3M
wiki_link                https://en.wikipedia.org/wiki/3M
Name: 0, dtype: object
1 Symbol                                                         AOS
Security                                               A. O. Smith
GICS Sector                                            Industrials
GICS Sub-Industry                                Building Products
Headquarters Location                         Milwaukee, Wisconsin
Date added                                              20